In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pympi
import seaborn as sns
from scipy import interpolate

In [2]:
#Sample of coding postures for one task one visit. All other visits were done the same way just by changing directory of data

rootdir_glob = 'C:\\Users\\1\\elan_analysis\\data\\postures\\free_play\\1' 

In [3]:
listOfPostures = {}
totalSeconds = {}
codingFrequency = {}
all_annotations = {}
timeCoded = {}
totalLength = {}


In [4]:
for i, file in enumerate(glob.glob(os.path.join(rootdir_glob, '**'), recursive=True)):
    if os.path.isfile(os.path.join(rootdir_glob, file)):

        
        
        if file.split('.')[1] == 'eaf':
            file_name = file.split('\\')[-1][:12]
            visit = file.split('\\')[-1][6:7]
            task = file.split('\\')[-1][8:12]
            elan_file = pympi.Elan.Eaf(file)
            df = pd.DataFrame(columns=['id', 'label', 'StartTime', 'EndTime','Duration', 'Visit', 'Task'])
            for tier in elan_file.get_tier_names():
                for ann in elan_file.get_annotation_data_for_tier(tier):
                    df2 = pd.DataFrame({'id': file_name, 'label': tier, 'StartTime': ann[0]/1000, 'EndTime': ann[1]/1000, 'Duration': ann[1]/1000 - ann[0]/1000, 'Visit': visit, 'Task' : task}, index=[0])
                    df = pd.concat([df, df2], ignore_index=True)
                    df = df.sort_values('StartTime')
                    df['label'] = np.where((df.label == 'Supported sitting by caregiver') | (df.label == 'Supported sitting by hand') | (df.label == 'Reclined'), 'Sitting', df.label)
                    df['label'] = np.where((df.label == 'Pull to stand') | (df.label == 'Supported stand')| (df.label == 'Walking'), 'Standing upright', df.label)
                    df['label'] = np.where((df.label == 'Crawling'), 'Hands and knees', df.label)
                    all_annotations[file_name] = df.sort_values('StartTime').reset_index(drop=True)
                    listOfPostures[file_name] = all_annotations[file_name].loc[:, 'label'].unique()
    

In [5]:
for i, file in enumerate(glob.glob(os.path.join(rootdir_glob, '**'), recursive=True)):
    if os.path.isfile(os.path.join(rootdir_glob, file)):
        if file.split('.')[1] == 'txt':
            file_name_txt = file.split('\\')[-1][:12] 
            with open(file) as f:
                df = f.readline()
                
                totalSeconds[file_name_txt] = float(df.split('duration: ')[1].split('/ ')[1])
                codingFrequency[file_name_txt] = float(df.split('sample: ')[1].split('"')[0])/1000
                totalLength[file_name_txt] = round(totalSeconds[file_name_txt] / codingFrequency[file_name_txt])
                
                

In [6]:
#Transforming to 1-0
for file_name in all_annotations:
    timeSeriesCoded = pd.DataFrame(data=np.zeros([len(listOfPostures[file_name]), totalLength[file_name]]),
                                   index=listOfPostures[file_name],
                                   columns=range(totalLength[file_name]))

    for behaviour in listOfPostures[file_name]:
        for iValue in range(len(all_annotations[file_name][all_annotations[file_name]['label'] == behaviour].loc[:, 'StartTime'].reset_index(drop=True))):
            if all_annotations[file_name][all_annotations[file_name]['label'] == behaviour].loc[:, 'StartTime'].reset_index(drop=True)[iValue] == 0:
                timeSeriesCoded.loc[behaviour, :(round(all_annotations[file_name][all_annotations[file_name]['label'] == behaviour].loc[:, 'EndTime'].reset_index(drop=True)[iValue] / codingFrequency[file_name]))] = 1
            else:
                timeSeriesCoded.loc[behaviour,(round(all_annotations[file_name][all_annotations[file_name]['label'] == behaviour].loc[:, 'StartTime'].reset_index(drop=True)[iValue] / codingFrequency[file_name])):(round(all_annotations[file_name][all_annotations[file_name]['label'] == behaviour].loc[:, 'EndTime'].reset_index(drop=True)[iValue] / codingFrequency[file_name]))] = 1
    timeCoded[file_name] = timeSeriesCoded.transpose().astype(int)

In [ ]:
timeCoded

Interpolation

In [8]:
%store -r totalLength_vocs_free_1
interpolated_free_visit_1 = {}

In [9]:
for (key,value), (ids,time) in zip(timeCoded.items(), totalLength_vocs_free_1.items()):
    if len(value.columns) > 1:
        f = interpolate.interp2d(x = np.arange(0, len(value.columns),1), y = np.arange(0, value.shape[0], 1), z = value.astype(int))
        r = f(np.arange(0, value.shape[1],1), np.linspace(start = 0, stop = value.shape[0], num = time))
        test = pd.DataFrame(columns = value.columns, data = r)
        test[(test > 0.5)] = 1
        test[(test < 0.5)] = 0
        interpolated_free_visit_1[key] = test
        
    else:
        f = interpolate.interp1d(x = np.arange(0, value.shape[0], 1), y = value.astype(int).squeeze())
        r = f(np.linspace(start = 0, stop = value.shape[0] -1, num = time))
        test = pd.DataFrame(columns = value.columns, data = r)
        test[(test > 0.5)] = 1
        test[(test < 0.5)] = 0
        interpolated_free_visit_1[key] = test

In [10]:
%store interpolated_free_visit_1

Stored 'interpolated_free_visit_4_2' (dict)
